In [1]:
from MFEA_lib.tasks.function import Sphere, Ackley, Rosenbrock
from MFEA_lib.model.mfea import MFEA1
from MFEA_lib.operators.CrossOver import SBX_CrossOver
from MFEA_lib.operators.Selection import ElitismSelection
from MFEA_lib.operators.Mutation import Polynomial_Mutation
from MFEA_lib.model.utils import saveModel, loadModel, compareModel
from MFEA_lib.tasks.Benchmark import CEC17_benchmark, GECCO20_benchmark_50tasks


In [2]:
tasks = [Sphere(30, bound= [-100, 100]), 
        Ackley(50, bound= [-100, 100]), 
        Rosenbrock(20, bound= [-100, 100]), 
        Rosenbrock(30, bound= [-100, 100])
]


In [3]:
model = MFEA1()
model.compile(
    cross_over= SBX_CrossOver(nc = 15),
    mutation= Polynomial_Mutation(nm = 15),
    selection= ElitismSelection()
)
sol, his = model.fit(tasks= tasks, num_generations= 2000, num_inds_each_task= 100, rmp =0.3)

Epoch [100/2000], [=>                  ]   5% ,func_val: [198.1348853026179, 20.702411977236018, 26135.212508295124, 290986.8487014828]

Epoch [200/2000], [==>                 ]  10% ,func_val: [5.35061037733814, 20.266413207788204, 340.1238125759362, 791.8875691226968]

Epoch [300/2000], [===>                ]  15% ,func_val: [0.6532319040881828, 20.088210320541048, 50.65260567921965, 90.68826675807571]

Epoch [400/2000], [====>               ]  20% ,func_val: [0.08177051578309259, 20.03612856676901, 22.099192326628497, 34.86003172728698]

Epoch [500/2000], [=====>              ]  25% ,func_val: [0.034688485898128785, 20.018211524106075, 18.82300860850972, 31.4233941352708]

Epoch [600/2000], [======>             ]  30% ,func_val: [0.017550969781258238, 20.009876810637216, 18.082875011646166, 29.69556126375481]

Epoch [700/2000], [=======>            ]  35% ,func_val: [0.012055040175644318, 20.004442543575653, 17.704919721165975, 29.185040781365355]

Epoch [800/2000], [========>      

#### Benchmark
##### CEC17


In [4]:
cec17_10tasks = CEC17_benchmark.get_10tasks_benchmark()

In [5]:
model1 = MFEA1()
model1.compile(
    cross_over= SBX_CrossOver(nc = 2),
    mutation= Polynomial_Mutation(nm = 5),
    selection= ElitismSelection()
)

sol1, his1 = model1.fit(tasks= cec17_10tasks, num_generations= 1000, num_inds_each_task= 100, rmp =0.3, 
                        evaluate_initial_skillFactor= True, num_epochs_printed= 10)

saveModel(model1, "./RESULTS/cec17_mfea1.mso")

Epoch [100/1000], [==>                 ]  10% ,func_val: [9319.112673543843, 7605.98603080032, 11365.38833706203, 11.866980739158677, 49905595.326537855, 15.578755212626593, 38.92339672643716, 10632.530351667368, 3.4995351550560865, 2888.055684367791]

Epoch [200/1000], [====>               ]  20% ,func_val: [2187.491998454848, 1450.1756462372764, 2142.172351186129, 6.222122441665505, 1741556.147845116, 9.575588341247828, 24.87495059755811, 7591.559900582379, 1.5617363562510962, 879.9535267482858]

Epoch [300/1000], [======>             ]  30% ,func_val: [563.3074400051333, 326.26817724850434, 460.5553436381643, 4.822085274497084, 241204.46037691325, 6.172325060770191, 19.673403648604122, 5107.009087943759, 1.1021902245356485, 461.5296252408989]

Epoch [400/1000], [========>           ]  40% ,func_val: [160.0529716320153, 101.52494778217144, 154.45944089647108, 4.230332063023916, 23063.453654522367, 4.041645073800044, 17.938013346288912, 4122.230206579836, 0.9812278884588852, 341.27739

'Saved'

In [6]:
cec17_10tasks_fixed = CEC17_benchmark.get_10tasks_benchmark(fix= True)


In [7]:
model1 = MFEA1()
model1.compile(
    cross_over= SBX_CrossOver(nc = 2),
    mutation= Polynomial_Mutation(nm = 5),
    selection= ElitismSelection()
)

sol1, his1 = model1.fit(tasks= cec17_10tasks_fixed, num_generations= 1000, num_inds_each_task= 100, rmp =0.3, 
                        evaluate_initial_skillFactor= True, num_epochs_printed= 10)

saveModel(model1, "./RESULTS/cec17fixed_mfea1.mso")

Epoch [100/1000], [==>                 ]  10% ,func_val: [7139.558955078769, 10481.282276075282, 8782.852211178291, 10.431166019790389, 27575628.821651857, 17.32599210233359, 36.86000579688803, 11920.555819313115, 3.7129660205691777, 2438.826269531221]

Epoch [200/1000], [====>               ]  20% ,func_val: [1197.7972261294233, 1509.6516946766035, 1785.8167600036995, 5.413998908539355, 636105.354764569, 9.963924617363972, 23.722207542098204, 8956.425391502731, 1.5757674930443608, 856.9444690555442]

Epoch [300/1000], [======>             ]  30% ,func_val: [312.43095261536706, 330.39863073954643, 435.4324519911939, 4.515793315087436, 79789.47875282726, 5.853289718707135, 17.33801565976259, 6754.4115519489915, 1.130496015897766, 441.08216378408844]

Epoch [400/1000], [========>           ]  40% ,func_val: [105.43590229848313, 85.43085774332297, 84.20270220841387, 4.216902232153217, 16462.42057772374, 3.835396091572445, 14.75955575260042, 5868.7102622534185, 1.0196475642602945, 340.5375

'Saved'